In [16]:
!pip -q install pyomo
from pyomo.environ import *
import numpy as np

**Answer 1 (formulaton part ) report card**

**2**. Create a model using the Pyomo framework to solve your formulation. Report the number of SKUs
transported (report only those nonzero values) from warehouses W to markets M. Also, report the
total cost for transportation of SKUs.

In [8]:
model=ConcreteModel()
m=8
n=8
row_indices =np.arange(m)
col_indices =np.arange(n)
Inventory =np.array([185, 350, 405,280,105,75,425,270])
demand = np.array([175 ,400, 550, 70, 650, 100, 65, 85])
cost = np.array([[627 ,617, 1270, 2072, 1415, 2943, 1064, 4265],[159, 1432, 351, 3372, 237, 3373 ,1456 ,5059 ],[609 ,1943, 77 ,3365 ,298, 3021, 1484 ,4803],[2143, 671 ,3876 ,537 ,1902 ,1941, 1682, 3348],[2742 ,2089, 1931 ,2199 ,1226 ,739 ,691, 1128],[2053 ,860 ,2995, 68 ,2069, 2065 ,993, 3342],[656 ,817 ,1245 ,2141 ,1534 ,3351, 1137, 3887],[195,1862 ,1331 ,2921, 402 ,2825, 563, 4441 ]])

model.x =Var(row_indices,col_indices,domain = NonNegativeIntegers)
#objective
model.obj =Objective(expr =sum(cost[i][j]*model.x[i,j] for i in row_indices for j in col_indices ),sense = minimize)
#constraint list
model.constraints = ConstraintList()
[model.constraints.add(expr =sum(model.x[i,j] for j in col_indices) <= Inventory[i] ) for i in row_indices]
[model.constraints.add(expr =sum(model.x[i,j] for i in row_indices) >= demand[j] ) for j in col_indices]
model.pprint()




4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
       

In [9]:
!apt-get install -y -qq coinor-cbc
optCBC  = SolverFactory('cbc')

In [10]:
resultCBC= optCBC.solve(model)
print(resultCBC)

def OptResults(model):
  print('\nobjective =' ,model.obj())
  print('\ndecision Variable')
  for i in row_indices:
    for j in col_indices:
      print('x[',i,']=',model.x[i,j].value)
  print('\nconstraints')
  model.constraints.display()
modelSolution = OptResults(model)



Problem: 
- Name: unknown
  Lower bound: 1166465.0
  Upper bound: 1166465.0
  Number of objectives: 1
  Number of constraints: 16
  Number of variables: 64
  Number of binary variables: 0
  Number of integer variables: 64
  Number of nonzeros: 64
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.03464198112487793
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


objective = 1166465.0

decision Variable
x[ 0 ]= 0.0
x[ 0 ]= 185.0
x[ 0 ]= 0.0
x[ 0 ]= 0.0
x[ 0 ]= 0.0
x[ 0 ]= 0.0
x[ 0 ]= 0.0
x[ 0 ]= 0.0
x[ 1 ]= 0.0
x[ 1 ]= 0.0
x[ 1 ]= 0.0
x[ 1 ]= 0.0
x[ 1 ]= 350.0
x[ 1 ]= 0.0
x[ 1 ]= 0.0


In [11]:
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(model)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


**3.** Suppose the W2−M1 link is disrupted, and no transportation is possible on the W2−M1 link. Without
changing the Pyomo model, how will you solve this problem? Report the new solution value and the
nonzero flows in the network. Comment on your observations.

In [17]:
# Fix the variable associated with the disrupted link (W2-M1) to zero
model.x[1, 0].fix(0)
SolverFactory('cbc').solve(model)
# Print the new solution value and nonzero flows in the network
total_cost = value(model.obj)
print("New Solution Value (Total Cost):", total_cost)

nonzero_flows = [(i, j, value(model.x[i, j])) for i in row_indices for j in col_indices if value(model.x[i, j]) > 0]
print("Nonzero Flows in the Network:")
for i, j, flow in nonzero_flows:
    print(f"Flow from {i} to {j}: {flow}")


New Solution Value (Total Cost): 1166465.0
Nonzero Flows in the Network:
Flow from 0 to 1: 185.0
Flow from 1 to 4: 350.0
Flow from 2 to 2: 375.0
Flow from 2 to 4: 30.0
Flow from 3 to 1: 200.0
Flow from 3 to 5: 80.0
Flow from 4 to 5: 20.0
Flow from 4 to 7: 85.0
Flow from 5 to 3: 70.0
Flow from 5 to 6: 5.0
Flow from 6 to 0: 175.0
Flow from 6 to 1: 15.0
Flow from 6 to 2: 175.0
Flow from 6 to 6: 60.0
Flow from 7 to 4: 270.0


**4.** Further in addition to W2 − M1 link, the W6 − M4 link and W5 − M8 links are disrupted and no
transportation is possible along these links as well. Report the new solution value and the nonzero
flows in the network. Explain your observations.

In [18]:
# Fix the variables associated with the disrupted links to zero
disrupted_links = [(1, 0), (5, 3), (4, 7)]
for link in disrupted_links:
    i, j = link
    model.x[i, j].fix(0)

# Re-solve the model
SolverFactory('cbc').solve(model)

# Print the new solution value and nonzero flows in the network
total_cost = value(model.obj)
print("New Solution Value (Total Cost):", total_cost)

nonzero_flows = [(i, j, value(model.x[i, j])) for i in row_indices for j in col_indices if value(model.x[i, j]) > 0]
print("Nonzero Flows in the Network:")
for i, j, flow in nonzero_flows:
    print(f"Flow from {i} to {j}: {flow}")


New Solution Value (Total Cost): 1289875.0
Nonzero Flows in the Network:
Flow from 0 to 1: 185.0
Flow from 1 to 4: 350.0
Flow from 2 to 2: 375.0
Flow from 2 to 4: 30.0
Flow from 3 to 1: 200.0
Flow from 3 to 3: 70.0
Flow from 3 to 7: 10.0
Flow from 4 to 5: 100.0
Flow from 4 to 6: 5.0
Flow from 5 to 7: 75.0
Flow from 6 to 0: 175.0
Flow from 6 to 1: 15.0
Flow from 6 to 2: 175.0
Flow from 6 to 6: 60.0
Flow from 7 to 4: 270.0
